## Imports:

In [ ]:
! pip install pycryptodome

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import csv
from Crypto.Util.number import getPrime
from Crypto.Util.number import inverse
from Crypto.Random.random import randrange
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from math import gcd
import os
import time
import numpy as np
from random import randbytes
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

## Reading the input files:





In [ ]:
def Reading_file(file_name):

  with open(file_name ,'rb') as file:
    lines = file.read()

  splitted_bytes = [lines[i:i+14] for i in range(0, len(lines), 14)]
  
  return (list(splitted_bytes))



In [ ]:
bytes_1KB= Reading_file('1KB_file.txt')


In [ ]:
bytes_5KB= Reading_file('5KB_file.txt')

In [ ]:
bytes_10KB= Reading_file('10KB_file.txt')

In [ ]:
bytes_100KB= Reading_file('100KB_file.txt')

## Generating the key:

In [ ]:
key= os.urandom(32)
print(key)

b'\x02\xcc\xb6\xdeR\xdfz\x00z)J\x82\xcc\x01\x0f\xae\xe6..\xd4uz\x88\xd7\xf1\xe2hs\xec\x8f\x91\x06'


## Generating the BS+2 random data:

In [ ]:
random_bytes = randbytes(14)
random_bytes[-2:]
added_2_bytes= random_bytes[-2:]
random_BS=random_bytes+added_2_bytes
print(random_BS)

b'\x9d\xf6w\x9b\x1a4\xecI=\x13\x9d\xe5\x97\x90\x97\x90'


## Concatenating the random bytes with the plaintext:

In [ ]:
bytes_1KB.insert(0,random_BS)
bytes_5KB.insert(0,random_BS)
bytes_10KB.insert(0,random_BS)
bytes_100KB.insert(0,random_BS)

## Encryption and decryption using AES:

In [ ]:
def AES_Encrypt(key,FR):

  
  E_cipher = AES.new(key, AES.MODE_ECB) # ECB mode
  modified_data = E_cipher.encrypt(FR)

  return modified_data


## ِAuxiliary Functions:

## Generating the Initial vector (IV):

In [ ]:
IV = bytearray(16)
print(IV)


bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')


In [ ]:
FR= IV  # feedback register is initially set to the initial vector

## XORing operation:

In [ ]:
def XOR(plaintext,ciphertext):
  
  # convert the input bytearrays to numpy arrays
  plaintext= np.frombuffer(plaintext, dtype=np.uint8)
  ciphertext = np.frombuffer(ciphertext, dtype=np.uint8)

  # XOR them element-wise and convert the result to a bytes object
  result = np.bitwise_xor(plaintext,ciphertext).tobytes()

  return result


## Updating the feedback register:

In [ ]:
def Register_update(FR, ciphertext):
  ciphertext = bytearray(ciphertext)
  FR = bytearray(FR)

  # Get the size of t
  size = len(ciphertext)

  # Shift c to the left by the size of t
  for i in range(size, len(FR)):
      FR[i - size] = FR[i]

  for i in range(len(FR) - size, len(FR)):
      FR[i] = ciphertext[i-len(FR)]

  # Convert the result back to bytes
  result = bytes(FR)

  return result


## Unpadding Function:

In [ ]:
def Unpadding(data):

  last_block= (data[-1])   # accessing the last block
  index= last_block.find(last_block[-1])  #accessing the last character (padding character)
  last_block= last_block[:index]   # removing the padding characters
  data[-1] =last_block    # updating the input data with the unpadded last block

  return data


## Main Function:

## PGP_CFB Mode:

In [ ]:
def PGP_CFBmode(plaintext,FR,key):

  ciphertext=[]
  BS= 14
 
  
  for i in range(len(plaintext)):
    
    # Handling the 1st case of PGP mode 16 bytes instead of 14 bytes 
      #  1. Handling the 14 bytes
    if i == 0:
     plain_block= plaintext[0]
     plain_block=plain_block[:BS]

     FRE= AES_Encrypt(key,FR)
     FRE= FRE [:BS]
     cipher_block1= XOR(plain_block,FRE)
      
      #  2. Handling the rest 2 bytes

     plain_block= plaintext[0]
     plain_block=plain_block[BS:BS+2]

     FRE= AES_Encrypt(key,FR)
     FRE= FRE [:2]
     cipher_block2= XOR(plain_block,FRE)

     # adding the 2 cipher blocks as they represent the 1st cipher block
     added_cipher=cipher_block1+cipher_block2
     ciphertext.append(added_cipher)


    else:
      if i == len(plaintext)-1 and len(plaintext[i])<14:  # flag is an indicator that the current block is the last block so it needs padding
        plain_block =pad(plaintext[i],BS)    #PKCS7 padding

      
      else:
        # take the current plain_block
        plain_block= plaintext[i]

      #encrypt the FR (feedback register):
      encrypted_FR= AES_Encrypt(key,FR)
      encrypted_FR= encrypted_FR[:BS] 

      # Xoring current plain_block with the encrypted_FR
      cipher_block= XOR(plain_block,encrypted_FR)
      ciphertext.append(cipher_block)

      #updating the FR using the current cipher_block 
      Register_update(FR,cipher_block)

      

  return ciphertext


## Evaluate the performance of the PGP-CFB mode for Encryption and Decryption:

## 1KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
cipher_1KB= PGP_CFBmode(bytes_1KB,FR,key)
end_time_encrypt = time.time() 
#print(cipher_1KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")

# Decryption part:

start_time_decrypt = time.time() 
decrypted_1KB= PGP_CFBmode(cipher_1KB,FR,key)
end_time_decypt = time.time()
#print(decrypted_1KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0095 seconds
Time taken for decryption:  0.0026 seconds


In [ ]:
print("Before Unpadding:",decrypted_1KB)
Unpadded_1KB= Unpadding(decrypted_1KB)
print("Before Unpadding:",Unpadded_1KB)


Before Unpadding: [b'\x9d\xf6w\x9b\x1a4\xecI=\x13\x9d\xe5\x97\x90\x97\x90', b'Victory seemed', b' unlikely. Mou', b'nt Saint Micha', b'el were two se', b'ts down and St', b'. Leo\xe2\x80\x99s cert', b'ainly had the ', b'upper hand in ', b'the third set,', b' dominating 9 ', b'points to 2. E', b'ach time St. L', b'eo\xe2\x80\x99s scored,', b' they pounded ', b'the ground in ', b'unison, furthe', b'r intimidating', b' an already sh', b'aky Claremorri', b's team. So how', b', in these cir', b'cumstances, di', b'd they snatch ', b'victory from t', b'he jaws of alm', b'ost certain de', b'feat?\r\n\r\n\r\n\r\n\r', b'\n\r\n\r\nThe first', b' set seemed to', b' suggest two t', b'eams who were ', b'equally matche', b'd. Michael\xe2\x80\x99s', b' fought point ', b'for point agai', b'nst a Leo\xe2\x80\x99s ', b'team whose def', b'ence was almos', b't impenetrable', b', yet Edel Nol', b'an managed to ', b'hit home some ', b'impressive spi', b'kes. However, ', b'as the set dre', b'w to a close, ', 

In [ ]:
with open('output_1KB.txt', 'wb') as f:

    # write the bytes object to the file
   for i in range(1,len(Unpadded_1KB)):
     decrypted_data = Unpadded_1KB[i]
     f.write(decrypted_data)

## 5KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
cipher_5KB= PGP_CFBmode(bytes_5KB,FR,key)
end_time_encrypt = time.time() 
#print(cipher_5KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
decrypted_5KB= PGP_CFBmode(cipher_5KB,FR,key)
end_time_decypt = time.time()
#print(decrypted_5KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0374 seconds
Time taken for decryption:  0.0346 seconds


In [ ]:
Unpadded_5KB= Unpadding(decrypted_5KB)
print(Unpadded_5KB)

[b'\x9d\xf6w\x9b\x1a4\xecI=\x13\x9d\xe5\x97\x90\x97\x90', b'Victory seemed', b' unlikely. Mou', b'nt Saint Micha', b'el were two se', b'ts down and St', b'. Leo\xe2\x80\x99s cert', b'ainly had the ', b'upper hand in ', b'the third set,', b' dominating 9 ', b'points to 2. E', b'ach time St. L', b'eo\xe2\x80\x99s scored,', b' they pounded ', b'the ground in ', b'unison, furthe', b'r intimidating', b' an already sh', b'aky Claremorri', b's team. So how', b', in these cir', b'cumstances, di', b'd they snatch ', b'victory from t', b'he jaws of alm', b'ost certain de', b'feat?\r\n\r\n\r\n\r\n\r', b'\n\r\nAs it final', b'ly dawned that', b' Claremorris c', b'ould actually ', b'win this match', b', the exciteme', b'nt and tension', b' in the arena ', b'reached fever ', b'pitch, and the', b' daddies on th', b'e sideline (Ge', b'rry Croghan & ', b'Con Nolan) ril', b'ed up the crow', b'd, ably assist', b'ed by vice-pri', b'ncipal Margare', b't Burke. Howev', b'er, St Leo\xe2\x80\x99s', b' finally

In [ ]:
with open('output_5KB.txt', 'wb') as f:

    # write the bytes object to the file
   for i in range(1,len(Unpadded_5KB)):
     decrypted_data = Unpadded_5KB[i]
     f.write( decrypted_data)

## 10KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
cipher_10KB= PGP_CFBmode(bytes_10KB,FR,key)
end_time_encrypt = time.time() 
#print(cipher_10KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
decrypted_10KB= PGP_CFBmode(cipher_10KB,FR,key)
end_time_decypt = time.time()
#print(decrypted_10KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0458 seconds
Time taken for decryption:  0.0729 seconds


In [ ]:
Unpadded_10KB= Unpadding(decrypted_10KB)
print(Unpadded_10KB)

[b'\x9d\xf6w\x9b\x1a4\xecI=\x13\x9d\xe5\x97\x90\x97\x90', b'Victory seemed', b' unlikely. Mou', b'nt Saint Micha', b'el were two se', b'ts down and St', b'. Leo\xe2\x80\x99s cert', b'ainly had the ', b'upper hand in ', b'the third set,', b' dominating 9 ', b'points to 2. E', b'ach time St. L', b'eo\xe2\x80\x99s scored,', b' they pounded ', b'the ground in ', b'unison, furthe', b'r intimidating', b' an already sh', b'aky Claremorri', b's team. So how', b', in these cir', b'cumstances, di', b'd they snatch ', b'victory from t', b'he jaws of alm', b'ost certain de', b'feat?\r\n\r\nThe f', b'irst set seeme', b'd to suggest t', b'wo teams who w', b'ere equally ma', b'tched. Michael', b'\xe2\x80\x99s fought po', b'int for point ', b'against a Leo\xe2', b'\x80\x99s team whose', b' defence was a', b'lmost impenetr', b'able, yet Edel', b' Nolan managed', b' to hit home s', b'ome impressive', b' spikes. Howev', b'er, as the set', b' drew to a clo', b'se, they found', b' themselves un', b'able to

In [ ]:
with open('output_10KB.txt', 'wb') as f:

    # write the bytes object to the file
   for i in range(1,len(Unpadded_10KB)):
     decrypted_data = Unpadded_10KB[i]
     f.write(decrypted_data)

## 100KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
cipher_100KB= PGP_CFBmode(bytes_100KB,FR,key)
end_time_encrypt = time.time() 
#print(cipher_100KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
decrypted_100KB= PGP_CFBmode(cipher_100KB,FR,key)
end_time_decypt = time.time()
#print(decrypted_100KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.4814 seconds
Time taken for decryption:  0.2215 seconds


In [ ]:
Unpadded_100KB= Unpadding(decrypted_100KB)
print(Unpadded_100KB)

[b'\x9d\xf6w\x9b\x1a4\xecI=\x13\x9d\xe5\x97\x90\x97\x90', b'Victory seemed', b' unlikely. Mou', b'nt Saint Micha', b'el were two se', b'ts down and St', b'. Leo\xe2\x80\x99s cert', b'ainly had the ', b'upper hand in ', b'the third set,', b' dominating 9 ', b'points to 2. E', b'ach time St. L', b'eo\xe2\x80\x99s scored,', b' they pounded ', b'the ground in ', b'unison, furthe', b'r intimidating', b' an already sh', b'aky Claremorri', b's team. So how', b', in these cir', b'cumstances, di', b'd they snatch ', b'victory from t', b'he jaws of alm', b'ost certain de', b'feat?\r\n\r\nThe f', b'irst set seeme', b'd to suggest t', b'wo teams who w', b'ere equally ma', b'tched. Michael', b'\xe2\x80\x99s fought po', b'int for point ', b'against a Leo\xe2', b'\x80\x99s team whose', b' defence was a', b'lmost impenetr', b'able, yet Edel', b' Nolan managed', b' to hit home s', b'ome impressive', b' spikes. Howev', b'er, as the set', b' drew to a clo', b'se, they found', b' themselves un', b'able to

In [ ]:
with open('output_100KB.txt', 'wb') as f:

    # write the bytes object to the file
   for i in range(1,len(Unpadded_100KB)):
     decrypted_data = Unpadded_100KB[i]
     f.write(decrypted_data)

## Comparison with RSA using {1 KB, 5 KB} file:

In [ ]:
import os
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP

def encrypt_file(input_path, output_path, key):
    # Initialize cipher object
    cipher = PKCS1_OAEP.new(key)

    # Define chunk size
    chunk_size = 32 # 1 MB

    # Read input file in chunks and encrypt each chunk
    with open(input_path, 'rb') as in_file, open(output_path, 'wb') as out_file:
        while True:
            chunk = in_file.read(chunk_size)
            if len(chunk) == 0:
                break
            encrypted_chunk = cipher.encrypt(chunk)
            out_file.write(encrypted_chunk)

def decrypt_file(input_path, output_path, key):
    # Initialize cipher object
    cipher = PKCS1_OAEP.new(key)

    # Define chunk size
    chunk_size = 256  # 1 MB

    # Read input file in chunks and decrypt each chunk
    with open(input_path, 'rb') as in_file, open(output_path, 'wb') as out_file:
        while True:
            encrypted_chunk = in_file.read(chunk_size)
            if len(encrypted_chunk) == 0:
                break
            decrypted_chunk = cipher.decrypt(encrypted_chunk)
            out_file.write(decrypted_chunk)


## 1KB file:

In [ ]:
# Example usage
input_file = '1KB_file.txt'
encrypted_file = 'encrypted.bin'
decrypted_file = '1KB_decrypted_RSA.txt'

# Generate RSA key pair
key_RSA = RSA.generate(2048)

# Encrypt the input file
start_time_encrypt = time.time() 
encrypt_file(input_file, encrypted_file, key_RSA)
end_time_encrypt = time.time() 

# Decrypt the encrypted file
start_time_decrypt = time.time() 
decrypt_file(encrypted_file, decrypted_file, key_RSA)
end_time_decypt = time.time()

# Delete key object from memory
del key_RSA
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0614 seconds
Time taken for decryption:  0.0850 seconds


## 5KB file:

In [ ]:
# Example usage
input_file = '5KB_file.txt'
encrypted_file = '5B_encrypted.bin'
decrypted_file = '5KB_decrypted_RSA.txt'

# Generate RSA key pair
key_RSA = RSA.generate(2048)

# Encrypt the input file
start_time_encrypt = time.time() 
encrypt_file(input_file, encrypted_file, key_RSA)
end_time_encrypt = time.time() 

# Decrypt the encrypted file
start_time_decrypt = time.time() 
decrypt_file(encrypted_file, decrypted_file, key_RSA)
end_time_decypt = time.time()

# Delete key object from memory
del key_RSA
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.1949 seconds
Time taken for decryption:  0.4079 seconds


## Evaluate the performance of the standard CFB mode for Encryption and Decryption:

In [ ]:
def get_file_as_a_one_text(file_name):

  with open(file_name ,'rb') as file:
    lines = file.read()
  
  return (lines)

In [ ]:
file_text_1KB = get_file_as_a_one_text('1KB_file.txt')

In [ ]:
file_text_5KB = get_file_as_a_one_text('5KB_file.txt')

In [ ]:
file_text_10KB = get_file_as_a_one_text('10KB_file.txt')

In [ ]:
file_text_100KB = get_file_as_a_one_text('100KB_file.txt')

In [ ]:
from Crypto.Cipher import AES

def pad_iv(iv):
    """Pad the initialization vector to 16 bytes."""
    if len(iv) < 16:
        return iv.ljust(16, b'\x00')
    elif len(iv) > 16:
        return iv[:16]
    else:
        return iv

def cfb_encrypt(plaintext, iv, Key):
    
    iv = pad_iv(iv)
    cipher = AES.new(Key, AES.MODE_CFB, iv=iv, segment_size=128)
    ciphertext = cipher.encrypt(plaintext)
    return ciphertext

def cfb_decrypt(ciphertext, iv, Key):
  
    iv = pad_iv(iv)
    cipher = AES.new(Key, AES.MODE_CFB, iv=iv, segment_size=128)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext

## 1KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
standard_cipher_1KB = cfb_encrypt(file_text_1KB, IV, key)
end_time_encrypt = time.time() 
#print(cipher_100KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
standard_decrypted_1KB = cfb_decrypt(standard_cipher_1KB, IV, key)
end_time_decypt = time.time()
#print(decrypted_100KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0005 seconds
Time taken for decryption:  0.0003 seconds


In [ ]:
with open('standard_output_1KB.txt', 'wb') as f:
    # write the bytes object to the file
    for i in range(0, len(standard_decrypted_1KB), 1024):
        decrypted_data = standard_decrypted_1KB[i:i+1024]
        f.write(bytes(decrypted_data))

## 5KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
standard_cipher_5KB = cfb_encrypt(file_text_5KB, IV, key)
end_time_encrypt = time.time() 
#print(cipher_100KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
standard_decrypted_5KB = cfb_decrypt(standard_cipher_5KB, IV, key)
end_time_decypt = time.time()
#print(decrypted_100KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0004 seconds
Time taken for decryption:  0.0002 seconds


In [ ]:
with open('standard_output_5KB.txt', 'wb') as f:
    # write the bytes object to the file
    for i in range(0, len(standard_decrypted_5KB), 1024):
        decrypted_data = standard_decrypted_5KB[i:i+1024]
        f.write(bytes(decrypted_data))

## 10KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
standard_cipher_10KB = cfb_encrypt(file_text_10KB, IV, key)
end_time_encrypt = time.time() 
#print(cipher_100KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
standard_decrypted_10KB = cfb_decrypt(standard_cipher_10KB, IV, key)
end_time_decypt = time.time()
#print(decrypted_100KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0003 seconds
Time taken for decryption:  0.0002 seconds


In [ ]:
with open('standard_output_10KB.txt', 'wb') as f:
    # write the bytes object to the file
    for i in range(0, len(standard_decrypted_10KB), 1024):
        decrypted_data = standard_decrypted_10KB[i:i+1024]
        f.write(bytes(decrypted_data))

## 100KB file:

In [ ]:
# Encryption part: 

start_time_encrypt = time.time() 
standard_cipher_100KB = cfb_encrypt(file_text_100KB, IV, key)
end_time_encrypt = time.time() 
#print(cipher_100KB)
print("Time taken for encyption: ","{:.4f}".format(end_time_encrypt - start_time_encrypt), "seconds")


# Decryption part:

start_time_decrypt = time.time() 
standard_decrypted_100KB = cfb_decrypt(standard_cipher_100KB, IV, key)
end_time_decypt = time.time()
#print(decrypted_100KB)
print("Time taken for decryption: ","{:.4f}".format(end_time_decypt - start_time_decrypt), "seconds")

Time taken for encyption:  0.0015 seconds
Time taken for decryption:  0.0039 seconds


In [ ]:
with open('standard_output_100KB.txt', 'wb') as f:
    # write the bytes object to the file
    for i in range(0, len(standard_decrypted_100KB), 1024):
        decrypted_data = standard_decrypted_100KB[i:i+1024]
        f.write(bytes(decrypted_data))

In [ ]:
def standard_CFB_mode(plaintext, key, iv, mode):
    cipher = AES.new(key, AES.MODE_CFB, iv)
    if mode == 'encrypt':
        return cipher.encrypt(plaintext)
    elif mode == 'decrypt':
        return cipher.decrypt(plaintext)
    else:
        raise ValueError("Invalid mode: choose 'encrypt' or 'decrypt'")


# Encrypt and decrypt the files using the standard CFB mode and measure the time taken

file_sizes = [1, 5, 10, 100]  # KB
file_names = ['1KB_file.txt', '5KB_file.txt', '10KB_file.txt', '100KB_file.txt']

for file_size, file_name in zip(file_sizes, file_names):
    with open(file_name, 'rb') as f:
        plaintext = f.read()

    # Encrypt
  
    import os

    key = os.urandom(16)
    start_time_encrypt = time.time()
    ciphertext = standard_CFB_mode(plaintext, key, IV, 'encrypt')
    end_time_encrypt = time.time()
    print(f"Time taken for {file_size}KB file encryption: {end_time_encrypt - start_time_encrypt:.4f} seconds")

    # Decrypt
    start_time_decrypt = time.time()
    decrypted_text = standard_CFB_mode(ciphertext, key, IV, 'decrypt')
    end_time_decrypt = time.time()
    print(f"Time taken for {file_size}KB file decryption: {end_time_decrypt - start_time_decrypt:.4f} seconds")

Time taken for 1KB file encryption: 0.0002 seconds
Time taken for 1KB file decryption: 0.0001 seconds
Time taken for 5KB file encryption: 0.0004 seconds
Time taken for 5KB file decryption: 0.0004 seconds
Time taken for 10KB file encryption: 0.0007 seconds
Time taken for 10KB file decryption: 0.0006 seconds
Time taken for 100KB file encryption: 0.0072 seconds
Time taken for 100KB file decryption: 0.0045 seconds
